# Практика
Используемые библиотеки

In [59]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing
import random


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [60]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
#s = requests.get(url).content
#df=pd.read_csv(io.StringIO(s.decode('UTF8')))
### Если не работает загрузка on-line
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=',')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [61]:
class Predicate:

    predicate_type: str
    regexp_less_or_equal = r'не более (\d+(?:,\d+)?)(-\d+(?:,\d+)?)?'
    regexp_in_between = r'в пределах (\d+(?:,\d+)?)-(\d+(?:,\d+)?)'
    less_than_value: float
    left: float
    right: float

    def __init__(self, description: str):
        if 'в пределах'  in description:
            self.range(description)
        elif 'не более' in description:
            self.less_or_equal(description)
        else:
            self.predicate_type = 'unknown'
    
    def less_or_equal(self, less_or_equal_description):
        self.predicate_type = 'less_or_equal'
        arr = re.findall(self.regexp_less_or_equal, less_or_equal_description)
        res = arr[0][0]
        if '' !=arr[0][1]:
            # ugly shanenigans around 'не более 2-3' to take 3
            res = arr[0][1]
            res = res.replace('-','')
        if ',' in res:
            res = res.replace(',','.')
        self.less_than_value = float(res)
        


    def range(self, in_description):
        self.predicate_type = 'range'
        arr = re.search(self.regexp_in_between, in_description).groups()
        self.left = float(arr[0])
        self.right = float(arr[1])

    def check_value(self, value):
        if 'less_or_equal' == self.predicate_type:
            return value <= self.less_than_value
        if 'range' == self.predicate_type:
            return value >= self.left and value <= self.right
        raise Exception('Unknown predicate type')

new_column = list()

for a in df.iterrows():
    bb = a[1]
    try:
        value = float(bb['Результат анализа'])
    except:
        value = 0
    predicate = bb['Норматив']
    is_ok = Predicate(predicate).check_value(value)
    str_ok = 'Норма' if is_ok else 'Отклонение'
    new_column.append(str_ok) 

df['Интерпретация'] = new_column   
df.set_index('Интерпретация', inplace=True) 
assert 'Интерпретация' == df.index.name
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
Интерпретация,,,,
Норма,pH,единицы pH,8.4,в пределах 6-9
Норма,Запах,баллы,1,не более 2-3
Норма,Цветность,градусы,б/цвета,не более 30
Норма,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
Норма,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
Норма,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
Норма,Нитраты (по NO3),мг/дм3,24,не более 45
Норма,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
Норма,Хлориды (Cl),мг/дм3,200,не более 350


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [62]:
n = 100000
apples_count = 4
orange_count = 5
def fill_the_box():
    fruits_list = list()
    for a in range(0, apples_count, 1):
        fruits_list.append('apple')
    for a in range(0, orange_count, 1):
        fruits_list.append('orange')
    return fruits_list

fruits_box = fill_the_box()
success_orange_bingos = 0

for i in range(n):
    box_for_iteration = fruits_box.copy()
    choice_list = list()
    choice_in_progress = True
    while choice_in_progress:
        #выбираем рандомно три фрукта. при этом игнорируем если при выборе попадает ранее выбранный фркут
        #(считаем что из ящика ранее выбранный фрукт уже вытащили)
        index = random.randint(0, len(box_for_iteration)-1)
        if index not in choice_list:
            choice_list.append(index)
        if 3 == len(choice_list):
            choice_in_progress = False
    orange_bingo = box_for_iteration[choice_list[0]] == 'orange' and  box_for_iteration[choice_list[1]] == 'orange' and box_for_iteration[choice_list[2]] == 'orange'
    if orange_bingo:
        success_orange_bingos+=1

print(f'Chances to peak three oranges in a row are:  {success_orange_bingos/n}')



Chances to peak three oranges in a row are:  0.11939


Вывод: Эксперимент в целом подтверждает аналитические выводы из сети

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [63]:
# Ваше решение